# Debug Qiskit Runtime jobs
{/* cspell:ignore ZIIIII, IZIIII,IIZIII, IIIZII, IIIIZI, IIIIIZ, rdiff */}

Before submitting a resource-intensive Qiskit Runtime workload to execute on hardware, you can use the the Qiskit Runtime [`Neat` (Noisy Estimator Analyzer Tool)](/api/qiskit-ibm-runtime/qiskit_ibm_runtime.debug_tools.Neat#neat) class to verify that your Estimator workload is set up correctly, is likely to return  accurate results, uses the most appropriate options for the specified problem, and more.

`Neat` Cliffordizes the input circuits for efficient simulation, while retaining its structure and depth. Clifford circuits suffer similar levels of noise and are a good proxy for studying the original circuit of interest.


The following examples illustrate situations where `Neat` can be a useful resource.

## Define the estimation problem

First, import the relevant packages and [authenticate to the Qiskit Runtime service.](/guides/setup-channel)

In [24]:
import numpy as np
import random

from qiskit.circuit import QuantumCircuit
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.quantum_info import SparsePauliOp

from qiskit_ibm_runtime import QiskitRuntimeService, EstimatorV2 as Estimator
from qiskit_ibm_runtime.debug_tools import Neat

from qiskit_aer.noise import NoiseModel, depolarizing_error

In [38]:
# Choose the least busy backend
service = QiskitRuntimeService()
backend = service.least_busy(operational=True, simulator=False)

# Generate a preset pass manager
pm = generate_preset_pass_manager(backend=backend, optimization_level=0)

# Set the random seed
random.seed(10)

## Initialize a target circuit

Consider a six-qubit circuit that has the following properties:

* Alternates random `RZ` rotations to layers of `CNOT` gates.
* Has a mirror structure, that is, it applies a unitary `U` followed by its inverse.

In [40]:
def generate_circuit(n_qubits, n_layers):
    r"""
    A function to generate a pseudo-random a circuit with ``n_qubits`` qubits and
    ``2*n_layers`` entangling layers of the type used in this notebook.
    """
    # An array of random angles
    angles = [
        [random.random() for q in range(n_qubits)] for s in range(n_layers)
    ]

    qc = QuantumCircuit(n_qubits)
    qubits = list(range(n_qubits))

    # do random circuit
    for layer in range(n_layers):
        # rotations
        for q_idx, qubit in enumerate(qubits):
            qc.rz(angles[layer][q_idx], qubit)

        # cx gates
        control_qubits = (
            qubits[::2] if layer % 2 == 0 else qubits[1 : n_qubits - 1 : 2]
        )
        for qubit in control_qubits:
            qc.cx(qubit, qubit + 1)

    # undo random circuit
    for layer in range(n_layers)[::-1]:
        # cx gates
        control_qubits = (
            qubits[::2] if layer % 2 == 0 else qubits[1 : n_qubits - 1 : 2]
        )
        for qubit in control_qubits:
            qc.cx(qubit, qubit + 1)

        # rotations
        for q_idx, qubit in enumerate(qubits):
            qc.rz(-angles[layer][q_idx], qubit)

    return pm.run(qc)

# Generate a random circuit
qc = generate_circuit(6, 3)
isa_qc = pm.run(qc)

qc.draw("mpl", idle_wires=0)

Choose single-Pauli `Z` errors as observables and use them to initialize the primitive unified blocs (PUBs).

In [41]:
# Initialize the observables
obs = ["ZIIIII", "IZIIII", "IIZIII", "IIIZII", "IIIIZI", "IIIIIZ"]
print(f"Observables: {obs}")

# Map the observables to the backend's layout
isa_obs = [SparsePauliOp(o).apply_layout(isa_qc.layout) for o in obs]

# Initialize the PUBs, which consist of six-qubit circuits with `n_layers` 1, ..., 6
all_n_layers = [1, 2, 3, 4, 5, 6]

pubs = [(generate_circuit(6, n) for n in all_n_layers)]

Observables: ['ZIIIII', 'IZIIII', 'IIZIII', 'IIIZII', 'IIIIZI', 'IIIIIZ']


## Cliffordize the circuits

The previously defined PUB circuits are not Clifford, which makes them difficult to simulate classically. However, you can use the `Neat` [`to_clifford`](/api/qiskit-ibm-runtime/qiskit_ibm_runtime.debug_tools.Neat#to_clifford) method to map tham to Clifford circuits for more efficient simulation.  The [`to_clifford`](/api/qiskit-ibm-runtime/qiskit_ibm_runtime.debug_tools.Neat#to_clifford) method is a wrapper around the [`ConvertISAToClifford`](/api/qiskit-ibm-runtime/qiskit_ibm_runtime.transpiler.passes.ConvertISAToClifford) transpiler pass. In particular, it replaces non-Clifford single-qubit gates in the original circuit with Clifford single-qubit gates, but it does not mutate the two-qubit gates, number of qubits, and circuit depth. 

The simulation can be done in both ideal ([`ideal_sim`](/api/qiskit-ibm-runtime/qiskit_ibm_runtime.debug_tools.Neat#ideal_sim)) and noisy ([`noisy_sim`](/api/qiskit-ibm-runtime/qiskit_ibm_runtime.debug_tools.Neat#noisy_sim)) conditions. The simulated results support mathematical operations, and can therefore be compared with each other (or with experimental results) to calculate figures of merit.  

See [Efficient simulation of stabilizer circuits with Qiskit Aer primitives](/guides/simulate-stabilizer-circuits) for more information about Clifford circuit simulation.

First, initialize `Neat`.

In [42]:
# You could specify a custom `NoiseModel` here. If `None`, `Neat`
# pulls the noise model from the given backend
noise_model = None

# Initialize `Neat`
analyzer = Neat(backend, noise_model)

Next, Cliffordize the PUBs.

In [43]:
clifford_pubs = analyzer.to_clifford(pubs)

clifford_pubs[0].circuit.draw("mpl", idle_wires=0)

## Application 1: Analyze the impact of noise on the circuit outputs

This example shows how to use `Neat` objects to study the impact of different noise models on PUBs as a function of circuit depth. This can be useful to set up expectations on the quality of the experimental results before running a job on a QPU. To learn more, see [Exact and noisy simulation with Qiskit Aer primitives.](/guides/simulate-with-qiskit-aer#exact-and-noisy-simulation-with-qiskit-aer-primitives)

Begin by performing ideal and noisy classical simulations.

In [44]:
# Perform a noiseless simulation
ideal_results = analyzer.ideal_sim(clifford_pubs)
print(f"Ideal results:\n {ideal_results}\n")

# Perform a noisy simulation with the backend's noise model
noisy_results = analyzer.noisy_sim(clifford_pubs)
print(f"Noisy results:\n {noisy_results}\n")

Ideal results:
 NeatResult([NeatPubResult(vals=array([1., 1., 1., 1., 1., 1.])), NeatPubResult(vals=array([1., 1., 1., 1., 1., 1.])), NeatPubResult(vals=array([1., 1., 1., 1., 1., 1.])), NeatPubResult(vals=array([1., 1., 1., 1., 1., 1.])), NeatPubResult(vals=array([1., 1., 1., 1., 1., 1.]))])

Noisy results:
 NeatResult([NeatPubResult(vals=array([0.96484375, 0.9765625 , 0.93359375, 0.93164062, 0.9921875 ,
       0.98828125])), NeatPubResult(vals=array([0.95703125, 0.953125  , 0.921875  , 0.93554688, 0.98242188,
       0.99414062])), NeatPubResult(vals=array([0.88476562, 0.89257812, 0.87695312, 0.87890625, 0.97265625,
       0.984375  ])), NeatPubResult(vals=array([0.82617188, 0.82617188, 0.78515625, 0.82617188, 0.9453125 ,
       0.97460938])), NeatPubResult(vals=array([0.77734375, 0.78515625, 0.7265625 , 0.796875  , 0.9375    ,
       0.96289062]))])



These results support mathematical operators and can be used to compute figures of merit.
The remainder of the guide uses the relative difference as a figure of merit to compare ideal results with noisy or experimental results, but similar figures of merit can be set up in a similar way.

The relative difference shows that the impact of noise grows with the circuits' sizes.

In [45]:
# Figure of merit: Relative difference
def rdiff(res1, re2):
    r"""The relative difference between `res1` and re2`.

    --> The closer to `0`, the better.
    """
    d = abs(res1 - re2) / res1
    return np.round(d.vals * 100, 2)


for idx, (ideal_res, noisy_res) in enumerate(
    zip(ideal_results, noisy_results)
):
    vals = rdiff(ideal_res, noisy_res)

    # Print the mean relative difference for the observables
    mean_vals = np.round(np.mean(vals), 2)
    print(
        f"Mean relative difference between ideal and noisy results for circuits with {all_n_layers[idx]}:\n  {mean_vals}%\n"
    )

Mean rel. diff. between ideal and noisy results:
  3.55%

Mean rel. diff. between ideal and noisy results:
  4.27%

Mean rel. diff. between ideal and noisy results:
  8.49%

Mean rel. diff. between ideal and noisy results:
  13.6%

Mean rel. diff. between ideal and noisy results:
  16.89%



You can specify different noise models in the analyzer. For example, you can study the impact of depolarizing noise on your PUBs.

In [46]:
# Set up a noise model with strength 0.02 on every two-qubit gate
noise_model = NoiseModel()
for qubits in backend.coupling_map:
    noise_model.add_quantum_error(
        depolarizing_error(0.02, 2), ["ecr", "cx"], qubits
    )

# Update the analyzer's noise model
analyzer.noise_model = noise_model

# Perform a noiseless simulation
ideal_results = analyzer.ideal_sim(clifford_pubs)

# Perform a noisy simulation with the backend's noise model
noisy_results = analyzer.noisy_sim(clifford_pubs)

# Compare the results
for idx, (ideal_res, noisy_res) in enumerate(
    zip(ideal_results, noisy_results)
):
    values = rdiff(ideal_res, noisy_res)

    # Print the mean relative difference for the observables
    mean_values = np.round(np.mean(values), 2)
    print(
        f"Mean relative difference between ideal and noisy results for circuits with {all_n_layers[idx]}:\n  {mean_values}%\n"
    )

Mean rel. diff. between ideal and noisy results:
  4.1%

Mean rel. diff. between ideal and noisy results:
  7.88%

Mean rel. diff. between ideal and noisy results:
  14.16%

Mean rel. diff. between ideal and noisy results:
  17.22%

Mean rel. diff. between ideal and noisy results:
  22.82%



As shown, given a noise model, you can try to quantify the impact of noise on the (Cliffordized version of the) PUBs of interest before running them on a QPU.

## Application 2: Benchmark different strategies

This example uses `Neat` to help identify the best options for your PUBs. To do so, consider running an estimation problem with [Probabilistic Error Amplification (PEA)](/guides/error-mitigation-and-suppression-techniques#probabilistic-error-amplification-pea). Which noise amplification factors will  work best?

In [47]:
# Generate a circuit with six qubits and six layers
qc = generate_circuit(6, 3)

# Use the same observables as previously
pubs = [(qc, isa_obs)]
clifford_pubs = analyzer.to_clifford(pubs)

In [48]:
noise_factors = [
    [1, 1.1],
    [1, 1.1, 1.2],
    [1, 1.5, 2],
    [1, 1.5, 2, 2.5, 3],
    [1, 4],
]

In [49]:
# Run the PUBs on a QPU
estimator = Estimator(backend)
estimator.options.default_shots = 100000
estimator.options.twirling.enable_gates = True
estimator.options.twirling.enable_measure = True
estimator.options.twirling.shots_per_randomization = 100
estimator.options.resilience.measure_mitigation = True
estimator.options.resilience.zne_mitigation = True
estimator.options.resilience.zne.amplifier = "pea"

jobs = []
for factors in noise_factors:
    estimator.options.resilience.zne.noise_factors = factors
    jobs.append(estimator.run(clifford_pubs))

results = [job.result() for job in jobs]

In [50]:
# Perform a noiseless simulation
ideal_results = analyzer.ideal_sim(clifford_pubs)

In [51]:
# Look at the mean relative difference to quickly tell the best choice for your options
for factors, res in zip(noise_factors, results):
    d = rdiff(ideal_results[0], res[0])
    print(
        f"Mean relative difference for factors {factors}:\n  {np.round(np.mean(d), 2)}%\n"
    )

Mean rel. diff. for factors [1, 1.1]:
  76.88%

Mean rel. diff. for factors [1, 1.1, 1.2]:
  8.23%

Mean rel. diff. for factors [1, 1.5, 2]:
  17.5%

Mean rel. diff. for factors [1, 1.5, 2, 2.5, 3]:
  12.22%

Mean rel. diff. for factors [1, 4]:
  12.73%



The result with the smallest difference helps you decide which options to choose.